<a href="https://colab.research.google.com/github/eTcilopp/geek_brains/blob/main/044_machine_learning/hw04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание №4

### 1. Загрузите тренировочные и тестовые датасеты

In [46]:
!pip install --upgrade category_encoders -q

In [47]:
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [48]:
Xtrain = pd.read_csv("https://gbcdn.mrgcdn.ru/uploads/asset/6057157/attachment/17bebdd19f5c427c6d531716a940aab0.csv")
Xtest = pd.read_csv("https://gbcdn.mrgcdn.ru/uploads/asset/6057155/attachment/7a470cd26573e5cb97a79b474076b4c2.csv")

Проведем EDA

In [49]:
Xtrain.head(5)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,target
0,50,218.593930,273536,0.104575,4,0.445026,0.274531,0.444334,5.970149,0.300298,0.0,0,49,0.100000,0
1,32,276.771005,173314,0.224684,11,0.445026,0.439103,0.444334,5.970149,0.300298,0.0,0,60,0.244418,0
2,38,218.593930,28887,0.032491,7,0.445026,0.274531,0.444334,25.606721,0.300298,0.0,0,50,0.244418,0
3,19,218.593930,427862,0.179322,10,0.049127,0.044987,0.009499,25.606721,0.300298,0.0,0,35,0.244418,0
4,44,218.593930,109339,0.032491,7,0.098837,0.115721,0.066581,5.970149,0.113590,0.0,0,46,0.185185,0


In [50]:
Xtrain.shape

(7500, 15)

In [51]:
Xtrain.isna().sum()

,0
f1,0
f2,75
f3,0
f4,0
f5,0
f6,0
f7,1875
f8,0
f9,0
f10,0


Наблюдаем NA значения. Избавимся от них
<br>
Более вдумчивый подход невозможен, т.к. непонятно, что скрывается за обозначениями f..

In [52]:
Xtrain.dropna(inplace=True)
Xtest.dropna(inplace=True)

In [53]:
Xtrain.isna().sum()

,0
f1,0
f2,0
f3,0
f4,0
f5,0
f6,0
f7,0
f8,0
f9,0
f10,0


### 2. Оцените баланс классов в задаче
- Затем попытайтесь устно ответить на вопрос, можно ли использовать accuracy как метрику качества в задаче?

In [54]:
Xtrain['target'].value_counts()

,count
target,
0,4238
1,1329


In [55]:
Xtest['target'].value_counts()

,count
target,
0,1913
1,587


Наблюдаем дисбаланс классов: кейсы с target = 1 представлены значительно реже.<br>
В результате модель скорее всего будет хорошо предсказывать отрицательные кейсы и плохо - положительные.
<br>
Метрика Accuracy показывает соотношение правильных ответов и общее число ответов. При несбалансированности классов модель может показывать высокую Accuracy за счет "доминирующих" классов, при этом общая применимость модели будет сомнительной

### 3. Постройте baseline-модель:
- разбейте TrainData на тренировочные (Train) и тестовые данные (Test);
- обучите LogisticRegression и SVC с параметрами по умолчанию на тренировочных данных (Train);
- примените модели на тестовых данных (Test).

In [56]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

X_train = Xtrain.drop(columns=['target'])
y_train = Xtrain['target']
X_test = Xtest.drop(columns=['target'])
y_test = Xtest['target']

X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_test_train, X_test_test, y_test_train, y_test_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

svc_model = SVC()
svc_model.fit(X_train_train, y_train_train)
y_train_pred_svc = svc_model.predict(X_train_test)
y_test_pred_svc = svc_model.predict(X_test_test)

In [57]:
f1_score_train = f1_score(y_train_test, y_train_pred_svc)
f1_score_test = f1_score(y_test_test, y_test_pred_svc)

print(f"F1-score на тренировочных данных: {f1_score_train}")
print(f"F1-score на тестовых данных: {f1_score_test}")

F1-score на тренировочных данных: 0.05303030303030303
F1-score на тестовых данных: 0.05555555555555555


Получаем достаточно низкие значения F1: модель SVC плохо справляется с  задачей

### 4. Улучшите модели
Попробуйте улучшить качество обученных моделей:
- можете задавать class_weights;
- можете изменять параметры модели;
- можете вручную или при помощи методов Python генерировать новые признаки и/или удалять существующие.

Это самая важная и творческая часть задания. Проводите как можно больше экспериментов!

Проведите минимиум три эксперимента: для каждого типа модели минимум один эксперимент.

Попробуем улучшить модель. Для начала сбалансируем классы

In [58]:
svc_model_b = SVC(class_weight="balanced")
svc_model_b.fit(X_train_train, y_train_train)
y_train_pred_svc_b = svc_model_b.predict(X_train_test)
y_test_pred_svc_b = svc_model_b.predict(X_test_test)

In [59]:
f1_score_train_b = f1_score(y_train_test, y_train_pred_svc_b)
f1_score_test_b = f1_score(y_test_test, y_test_pred_svc_b)

print(f"F1-score на тренировочных данных (сбалансированные классы): {f1_score_train_b}")
print(f"F1-score на тестовых данных (сбалансированные классы): {f1_score_test_b}")

F1-score на тренировочных данных (сбалансированные классы): 0.23809523809523808
F1-score на тестовых данных (сбалансированные классы): 0.22033898305084745


Результаты подскочили с 0.05 до 0.22. Значительный прогресс!
<br>
Однако, не станем останавливаться на достигнутом и попробуем подобрать гиперпараметры модели

In [60]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

svc_model = SVC(class_weight="balanced", C=10)
svc_model.fit(X_train_train, y_train_train)
y_train_pred_svc = svc_model.predict(X_train_test)
y_test_pred_svc = svc_model.predict(X_test_test)

f1_score_train_b = f1_score(y_train_test, y_train_pred_svc)
f1_score_test_b = f1_score(y_test_test, y_test_pred_svc)

print(f"F1-score на тренировочных данных (сбалансированные классы): {f1_score_train_b}")
print(f"F1-score на тестовых данных (сбалансированные классы): {f1_score_test_b}")


F1-score на тренировочных данных (сбалансированные классы): 0.3333333333333333
F1-score на тестовых данных (сбалансированные классы): 0.3511450381679389


Увелилив параметр С (Regularization Parameter; Регулирует баланс между ошибками классификации и шириной маржина (запаса между классами)
<br>
Улучшили работу модели с 0.22 до 0.35!

Использование ядер linear, poly не дает никаких улучшений - оставляем ядро "по умолчанию" rbf

Разобъем данные на категориальные и непрерывные

Попробуем оптимизировать фичи

In [61]:
categorical_columns = ['f1', 'f5', 'f12', 'f13']  # Категориальные
numerical_columns = [category for category in X_train.columns if category not in categorical_columns]  # Непрерывные

In [62]:
X_categorical = X_train[categorical_columns]
X_numerical = X_train[numerical_columns]

Подберем лучшие категориальные параметры

In [63]:
from sklearn.feature_selection import SelectKBest, chi2

X_categorical = X_categorical.astype(int)

chi2_selector = SelectKBest(chi2, k=2)
X_categorical_kbest = chi2_selector.fit_transform(X_categorical, y_train)

selected_categorical_features = chi2_selector.get_support(indices=True)

Непрерывные признаки

In [64]:
from sklearn.feature_selection import SelectKBest, f_classif

f_selector = SelectKBest(f_classif, k=2)
X_numerical_kbest = f_selector.fit_transform(X_numerical, y_train)

selected_numerical_features = f_selector.get_support(indices=True)

Объединим признаки

In [65]:
X_selected = np.hstack([X_categorical_kbest, X_numerical_kbest])

In [66]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_selected, y_train, test_size=0.2, random_state=42)

model = SVC(class_weight="balanced", C=10)
model.fit(X_train_train, y_train_train)
y_pred_train_train = model.predict(X_train_train)
y_pred_train_test = model.predict(X_train_test)
f1_score_train = f1_score(y_train_train, y_pred_train_train)
f1_score_test = f1_score(y_train_test, y_pred_train_test)

print(f"F1-score на тренировочных данных (сбалансированные классы): {f1_score_train}")
print(f"F1-score на тестовых данных (сбалансированные классы): {f1_score_test}")

F1-score на тренировочных данных (сбалансированные классы): 0.4604269293924466
F1-score на тестовых данных (сбалансированные классы): 0.4385733157199472


Результаты F1-score улучшились с  0.35 до 0.44!

Попробуем поменять гиперпараметр gamma

In [67]:
for gamma in [0.01, 0.1, 1]:
    model_gamma = SVC(class_weight="balanced", C=10, gamma=gamma)
    model_gamma.fit(X_train_train, y_train_train)

    y_pred_train_test = model_gamma.predict(X_train_test)

    f1_score_test = f1_score(y_train_test, y_pred_train_test)
    print(f"Gamma: {gamma}; F1-score на тестовых данных (сбалансированные классы): {f1_score_test}")

Gamma: 0.01; F1-score на тестовых данных (сбалансированные классы): 0.603085553997195
Gamma: 0.1; F1-score на тестовых данных (сбалансированные классы): 0.6075949367088608
Gamma: 1; F1-score на тестовых данных (сбалансированные классы): 0.5955678670360111


Лучшие результаты получились при значении gamma 0.1

### 5. Оцените на отложенной выборке качество наилучшей модели
В пунктах 3 и 4 вы построили много разных моделей.

Возьмите ту, которая дала наилучшее качество на тестовых данных (Test). Примените её на отложенной выборке (TestData) и выведите на экран значение метрики f1.

Применим самую лучшую модель на отложенной выборке

In [68]:
X_categorical_test = X_test[categorical_columns].astype(int)
X_numerical_test = X_test[numerical_columns]

X_categorical_kbest_test = chi2_selector.fit_transform(X_categorical_test, y_test)
X_numerical_kbest = f_selector.fit_transform(X_numerical_test, y_test)

X_selected = np.hstack([X_categorical_kbest_test, X_numerical_kbest])

In [69]:
y_test_pred = model.predict(X_selected)

f1_score_test = f1_score(y_test, y_test_pred)

print(f"F1-score на отложенной выборке (сбалансированные классы): {f1_score_test}")

F1-score на отложенной выборке (сбалансированные классы): 0.3933671119299862


Результат хуже, чем на тренировочных выборках, но все еще лучше, чем изначальные 0.05

### 6. Выполните хитрый трюк
Часто смешивание различных моделей даёт улучшение итогового предсказания. Попробуйте смешать две лучшие модели по формуле:
$$pred_{final} = \alpha\cdot pred_1 + (1-\alpha)\cdot pred_2$$.

Значение $\alpha$ подберите в цикле по Test-выборке. Оцените качество на отложенной выборке.

Удалось ли добиться улучшения качества?

Для начала обучим модель model_gamma на тех же данных, что и модель model<br>
на отобранных признаках

In [70]:
# X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_selected, y_train, test_size=0.2, random_state=42)

model_gamma = SVC(class_weight="balanced", C=10, gamma=0.1)
model_gamma.fit(X_train_train, y_train_train)

model = SVC(class_weight="balanced", C=10)
model.fit(X_train_train, y_train_train)

SVC(C=10, class_weight='balanced')

In [71]:
model_1 = model
model_2 = model_gamma

for alpha in np.linspace(0, 1, 11):
    y_test_pred_1 = model_1.predict(X_selected)
    y_test_pred_2 = model_2.predict(X_selected)
    y_test_pred_final = alpha * y_test_pred_1 + (1 - alpha) * y_test_pred_2
    y_test_pred_final = (y_test_pred_final > 0.5).astype(int)
    f1_score_test = f1_score(y_test, y_test_pred_final)
    print(f"Alpha: {alpha}, F1-score на отложенной выборке: {f1_score_test}")

Alpha: 0.0, F1-score на отложенной выборке: 0.0
Alpha: 0.1, F1-score на отложенной выборке: 0.0
Alpha: 0.2, F1-score на отложенной выборке: 0.0
Alpha: 0.30000000000000004, F1-score на отложенной выборке: 0.0
Alpha: 0.4, F1-score на отложенной выборке: 0.0
Alpha: 0.5, F1-score на отложенной выборке: 0.0
Alpha: 0.6000000000000001, F1-score на отложенной выборке: 0.3933671119299862
Alpha: 0.7000000000000001, F1-score на отложенной выборке: 0.3933671119299862
Alpha: 0.8, F1-score на отложенной выборке: 0.3933671119299862
Alpha: 0.9, F1-score на отложенной выборке: 0.3933671119299862
Alpha: 1.0, F1-score на отложенной выборке: 0.3933671119299862


Результаты улучшить не удалось.<br>
Лучший результат выдала модель с уменьшенным числом параметров, сбалансированная и с gamma = 0.1<br>
на тестовой части исходной выборки.


Для оптимизации модели мы использовали:<br>
- сбалансировали классы;<br>
  - оптимизировали гиперпараметры:
  - Подобрали параметр С (Regularization Parameter);<br>
  - Подобрали параметр gamma;<br>
  - Использовали разные ядерные функции;<br>
  - Использовали "хитрый трюк" со смешиванием результатов работы модели;<br>
- оптимальным образом уменьшили размерность данных.
<br><br>
Максимального качества удалось добиться на тестовых данных на модели<br>
- со сбалансированными классами,<br>
- с оптимизированными гиперпараметрами,<br>
- с уменьшенной размерностью.<br>
<br><br>
На отложенных данных результаты получились хуже, что свидетельствует о переобучении модели. <br>
# Выводы
Оптимизация модели и данных работает и может значительно улучшить качество модели